In [1]:
# Import your custom Python scripts
import sys
sys.path.append('../scripts')  #

from feature_enginerring import function1
from feature_selection import function2
from cross_validation import function3
from model_runner import function4



ModuleNotFoundError: No module named 'feature_enginerring'

In [ ]:
# Call functions from your Python scripts
result1 = function1()
result2 = function2()
